In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [2]:
data = tf.keras.utils.image_dataset_from_directory('images')
num_classes = len(data.class_names)
# Scale the images
data = data.map(lambda x, y: (x / 255.0, y))
data_iterator = data.as_numpy_iterator()
images, labels = next(data_iterator)
print(images.max())

Found 18944 files belonging to 11 classes.
1.0


In [3]:
print(len(data))
train_size = int(0.7 * len(data))
val_size = int(0.15 * len(data))+1
test_size = int(0.15 * len(data))+1
assert train_size + val_size + test_size == len(data)

train_data = data.take(train_size)
test_data = data.skip(train_size).take(test_size)
val_data = data.skip(train_size + test_size).take(val_size)

assert len(train_data) == train_size
assert len(test_data) == test_size
assert len(val_data) == val_size

592


### Training the model


In [5]:
from tensorflow import keras
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np

In [16]:

# Extract labels and one-hot encode them
train_labels = np.array([label for _, label in train_data.unbatch()])
val_labels = np.array([label for _, label in val_data.unbatch()])
test_labels = np.array([label for _, label in test_data.unbatch()])

# One-hot encode the labels
train_labels_one_hot = to_categorical(train_labels, num_classes=num_classes)
val_labels_one_hot = to_categorical(val_labels, num_classes=num_classes)
test_labels_one_hot = to_categorical(test_labels, num_classes=num_classes)

print(train_labels_one_hot.shape)
print(val_labels_one_hot.shape)
print(test_labels_one_hot.shape)


(13248, 11)
(2848, 11)
(2848, 11)


In [18]:

# Define the model
model = Sequential()
model.add(Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Use softmax for multi-class



In [19]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 30, 30, 16)       

In [20]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [15]:
history = model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[tensorboard_callback])

Epoch 1/20


ValueError: in user code:

    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, samp

In [9]:
model.evaluate(test_data)

38/38 [==============================] - 8s 88ms/step - loss: 4.0887 - accuracy: 0.4268


[4.088664531707764, 0.4268092215061188]